In [68]:
from twitterscraper import query_tweets
import tweepy
import datetime
import json, codecs
import jsonpickle
import GetOldTweets3 as got
import math

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# Load Twitter API credentials
# Credit to Temple Moore for authentication help and setting up tweepy

with open('./credentials.json') as creds:
    info = json.load(creds)
    consumer_key = info['CONSUMER_KEY']
    consumer_secret = info['CONSUMER_SECRET']
    access_token = info['ACCESS_TOKEN']
    access_secret = info['ACCESS_SECRET']

In [3]:
# Authenticate to Twitter

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Autentication Successful")
except:
    print("Error during authentication")

Autentication Successful


In [4]:
user = api.me()
print(user.name)
print(api.rate_limit_status()['resources']['search'])

DC DSI Client Project - Problem 1
{'/search/tweets': {'limit': 180, 'remaining': 180, 'reset': 1564442903}}


In [53]:
# Function written by using code modified from Jasmine Vasandani via Temple Moore
# With the help of got.manager, get all the tweets from specified timeframe, with specified query and search area
# default distance of 20km, default maxtweets of 1500
# Store all the collected tweets in DF

def twit_scrape(since, until, query, near, dist='20km', maxtweets=1500):
    tweets=[]
    final_df=pd.DataFrame()
    tweetCriteria = got.manager.TweetCriteria().setSince(since).setUntil(until).setQuerySearch(query).setNear(near).setWithin(dist).setMaxTweets(maxtweets).setLang('en')
    print(tweetCriteria)
    tweet = got.manager.TweetManager.getTweets(tweetCriteria)
    for i in range(len(tweet)):
        tweet_dict = {}
        tweet_dict['id'] = tweet[i].id
        tweet_dict['username'] = tweet[i].username
        tweet_dict['date'] = tweet[i].date
        tweet_dict['text'] = tweet[i].text
        tweet_dict['hashtags'] = tweet[i].hashtags
        tweet_dict['geo'] = tweet[i].geo
        tweet_dict['lat'] = near[0]
        tweet_dict['long'] = near[1]
        tweets.append(tweet_dict)

    df1=pd.DataFrame(tweets)
    final_df = pd.concat([df1.reset_index(), final_df.reset_index()], sort = True)
    print("Shape of DataFrame:", final_df.drop('index', axis = 1).shape)
    return final_df.drop('index', axis = 1)

In [54]:
# Testing the above function
df1 = twit_scrape(since='2019-07-04', until='2019-07-06', query='earthquake, OR quake',
            near=[34.0422, -118.2437], maxtweets = 25)

Shape of DataFrame: (25, 8)


In [55]:
df1.head()

,date,geo,hashtags,id,lat,long,text,username
0,2019-07-05 23:59:57+00:00,,,1147294071771160576,34.0422,-118.2437,Hopefully a devastating earthquake hits there!...,avvy73
1,2019-07-05 23:59:47+00:00,,,1147294029790339073,34.0422,-118.2437,"But, you can’t predict when/where they will st...",PatrickWalker
2,2019-07-05 23:59:38+00:00,,#Earthquake #sismo #Ridgecrest #California,1147293992406704134,34.0422,-118.2437,#Earthquake ( #sismo) M2.0 strikes 17 km NE of...,EMSC
3,2019-07-05 23:59:31+00:00,,,1147293966510915585,34.0422,-118.2437,I need to be fully dressed at all times till t...,Don__Richie
4,2019-07-05 23:59:24+00:00,,,1147293933740929024,34.0422,-118.2437,Are you talking about the earthquake?,AmmelZamora


In [69]:
df1

,date,geo,hashtags,id,lat,long,text,username
0,2019-07-05 23:59:57+00:00,,,1147294071771160576,34.0422,-118.2437,Hopefully a devastating earthquake hits there!...,avvy73
1,2019-07-05 23:59:47+00:00,,,1147294029790339073,34.0422,-118.2437,"But, you can’t predict when/where they will st...",PatrickWalker
2,2019-07-05 23:59:38+00:00,,#Earthquake #sismo #Ridgecrest #California,1147293992406704134,34.0422,-118.2437,#Earthquake ( #sismo) M2.0 strikes 17 km NE of...,EMSC
3,2019-07-05 23:59:31+00:00,,,1147293966510915585,34.0422,-118.2437,I need to be fully dressed at all times till t...,Don__Richie
4,2019-07-05 23:59:24+00:00,,,1147293933740929024,34.0422,-118.2437,Are you talking about the earthquake?,AmmelZamora
5,2019-07-05 23:59:21+00:00,,,1147293921866911744,34.0422,-118.2437,California said “you make my earthquake”,SongsHaven
6,2019-07-05 23:59:21+00:00,,#earthquake #Tsunami #pacificnorthwesthttps,1147293920533143552,34.0422,-118.2437,An incredible read. #earthquake #Tsunami #paci...,unrealfehr
7,2019-07-05 23:59:16+00:00,,,1147293901587271680,34.0422,-118.2437,"be careful, be alert. Califonia earthquake has...",hebsiba691
8,2019-07-05 23:59:09+00:00,,,1147293872554500098,34.0422,-118.2437,@AliceEvansGruff Uggggggh seismic scientists j...,HollywoodTed67
9,2019-07-05 23:59:06+00:00,,#TeamDowney #RobertDowneyJr #SusanDowney #BeWe...,1147293861028487169,34.0422,-118.2437,#TeamDowney #RobertDowneyJr #SusanDowney Hopin...,Fandango739


In [ ]:
def twit_scrape_grid(coords):
    for i in range(len(coords)):
        print(i)
        twit_scrape()

In [10]:
# This saves the dataframe to a csv - change the name for each individual scrape
# df1.to_csv('./harveyhouston_25mi.csv', index = False)